# **Cleaning the Data**

## Objectives

* Do initial exploration of data and clean it. There may be some preparation of the data as well.

## Inputs

* We will require the CSV files obtained in the Data Collection notebook.

## Outputs

* We will have cleaned versions of several of our CSV files.

## Additional Comments

* Remember to use Python 3.8.18.


---

# Change working directory

We need to change the working directory to the one containing the raw form of the CSV files.

In [1]:
import os
home_dir = '/workspace/pp5-ml-dashboard'
csv_dir ='/workspace/pp5-ml-dashboard/outputs/datasets/raw/csv' 
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)

/workspace/pp5-ml-dashboard


# Section 1: Initial Exploration

We have the following CSV files.

In [2]:
files = os.listdir(csv_dir)
for file in files:
    print(f"--{file}")
print(current_dir)

--game.csv
--line_score.csv
--other_stats.csv
--team_history.csv
/workspace/pp5-ml-dashboard


In the future, we may perform more extensive analysis which takes into account some of the data in the other CSV files. For the time being, these 4 files will be more than enough. (Probably too much even.)

First, let's look at what columns we have in `games.csv`. 

Note: The utility function `get_df` doesn't require the file extension and the default directory is the one containing the csv files.

In [3]:
import pandas as pd
from src.utils import get_df, get_info_df, info_dtype_dict

game_df = get_df('game')
print(game_df.shape)
game_df.head()

(65698, 55)


,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,2,0,Regular Season
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,25.0,51.0,-5,0,Regular Season
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,53.0,-6,0,Regular Season
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,22.0,47.0,-16,0,Regular Season
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,50.0,17,0,Regular Season


These games in the first few rows are very old. One consequence of this is that many of the common statistics of today were not tracked. One way to address this would be to impute the values, but the game of basketball evolved dramatically and so using the mean would not be very reflective of the actual game. We are missing a lot of data in these rows and so we will drop them. After all, the data frame contains data for approximately 65 thousand games.

Let's look at what columns are present.

In [4]:
game_df.columns

Index(['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home',
       'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home',
       'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home',
       'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home',
       'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home',
       'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away',
       'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away',
       'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away',
       'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away',
       'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away',
       'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away',
       'season_type'],
      dtype='object')

Many of these columns are redundant categorical data. For example:
- team_abbreviation_home
- team_name_home
- matchup_home
- video_available_home

as well as the away team versions, are not statistics that are relevant to what happened during the game. The team name even provides less information than expected as the name has changed over time, but the team_id does not.
# Attention
To Sean: did I actually do the below? or am I doing this? Edit this section appropriately nearer to the end of the project.
This is why we have kept the `team_history.csv` file, so that we can see what the team name is at various points in history. We could also keep the name columns and drop them at the last moment.

Also, we can drop any percentage based statistic as it will be completely determined by other features. For example, the columns `fg3m_home` and `fg3a_home` completely determine `fg3_pct_home`. So we will also remove any column containing the string `'_pct_'` in it.

In [5]:
game_drop_features = ['team_name_home','team_abbreviation_home','team_abbreviation_away','team_name_away','video_available_home','video_available_away', 'matchup_home','matchup_away']
pct_columns = [col for col in game_df.columns if '_pct_' in col]
game_drop_features.extend(pct_columns)

Consider the column `season_type`. It has the following unique values.

In [6]:
game_df['season_type'].unique()

array(['Regular Season', 'Playoffs', 'All-Star', 'All Star', 'Pre Season'],
      dtype=object)

Clearly, the All-Star games are out of place and don't represent a standard competition. Furthermore, the respective teams only play one game a season. Statistically, the number of games is negligible when compared to our the 1230 games played during the regular season.

In [7]:
all_star_games = game_df.query('season_type in ["All-Star", "All Star"]')
print(all_star_games.shape[0])
game_df = game_df.query('season_type != "All-Star"')
game_df = game_df.query('season_type != "All Star"')


128


There are also preseason games. These are sometimes played with teams outside of the league. Many veterans don't take the games seriously. It is also the case that there are exhibition matches with non-NBA teams that take place. So we remove these games from the dataset as well.

In [8]:
preseason = game_df.query('season_type == "Pre Season"')
print(preseason.shape[0])
preseason.team_name_away.unique()
game_df = game_df.query('season_type != "Pre Season"')

1536


We are primarily interested in Regular Season games, but we will leave the Play off games in for the time being.

Let's look back at the head of `game_df`.

In [9]:
game_df.head()

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away,season_type
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,2,0,Regular Season
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,25.0,51.0,-5,0,Regular Season
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,53.0,-6,0,Regular Season
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,22.0,47.0,-16,0,Regular Season
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,50.0,17,0,Regular Season


A lot of basic statistics, such as rebounds, are missing. Recall that the 3-point line wasn't introduced until 1979. The game of basketball has changed over the years. However, it probably changed most dramatically when it absorbed the ABA. Therefore, we will only consider games after the merger. 
# Attention
Or will I also get rid of games before the 3-point line. I don't think it is so crazy. It could be interesting to consider both. Then imputation will be slightly interesting as well.

In [10]:
from src.utils import add_cat_date

game_df = add_cat_date(game_df,'game_date')
print(game_df.shape)
game_df_after_1975 = game_df.query('Year >= 1975')
print(game_df.shape)
game_df.info()

(64034, 58)
(64034, 58)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 64034 entries, 0 to 65695
Data columns (total 58 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   season_id               64034 non-null  int64  
 1   team_id_home            64034 non-null  int64  
 2   team_abbreviation_home  64034 non-null  object 
 3   team_name_home          64034 non-null  object 
 4   game_id                 64034 non-null  int64  
 5   game_date               64034 non-null  object 
 6   matchup_home            64034 non-null  object 
 7   wl_home                 64034 non-null  object 
 8   min                     64034 non-null  int64  
 9   fgm_home                64021 non-null  float64
 10  fga_home                48587 non-null  float64
 11  fg_pct_home             48544 non-null  float64
 12  fg3m_home               50867 non-null  float64
 13  fg3a_home               45402 non-null  float64
 14  fg3_pct_home  

In [20]:
test_for_aba = game_df.query('Year == 1977')
teams = test_for_aba['team_name_home'].unique()
print(teams)

['Boston Celtics' 'Portland Trail Blazers' 'Golden State Warriors'
 'Washington Bullets' 'Detroit Pistons' 'Chicago Bulls'
 'San Antonio Spurs' 'Cleveland Cavaliers' 'Philadelphia 76ers'
 'Houston Rockets' 'Los Angeles Lakers' 'Denver Nuggets' 'New York Knicks'
 'Milwaukee Bucks' 'Phoenix Suns' 'Indiana Pacers' 'Buffalo Braves'
 'New Orleans Jazz' 'New Jersey Nets' 'Kansas City Kings'
 'Seattle SuperSonics' 'Atlanta Hawks']


We are finally ready to remove the columns discussed above.

In [12]:
'''try:
    game_df.drop(labels=game_drop_features, axis=1, inplace=True)
except Exception as e:
    print(str(e))
else:
    print("The columns have been successfully removed.")
'''

'try:\n    game_df.drop(labels=game_drop_features, axis=1, inplace=True)\nexcept Exception as e:\n    print(str(e))\nelse:\n    print("The columns have been successfully removed.")\n'

---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [13]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block (2852421808.py, line 5)